Note: Use this template to develop your project. Do not change the steps. For each step, you may add additional cells if needed.

#### Group Information

Group No: 

- Member 1:
- Member 2:
- Member 3:
- Member 4:

#### Import Libraries

In [100]:
%config Completer.use_jedi=False
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # only shows error messages

# import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd


#### Load the dataset

In [101]:
df = pd.read_csv('classification_dataset.csv')
df
# "f1", "f2", "f3", "f4" and "f5" determines "label" 


,f1,f2,f3,f4,f5,label
0,1.286233,15.643743,-1.879915,-11.294839,15.245472,0
1,2.853398,0.129878,17.620669,3.945204,8.157459,1
2,3.285310,3.176560,12.610554,-6.063613,1.831887,0
3,2.019516,-1.967793,9.306435,-0.938714,-1.203038,0
4,-2.326527,3.453234,13.855478,-5.236421,1.547216,0
...,...,...,...,...,...,...
995,-2.248262,-4.619586,3.248760,9.114543,4.370790,1
996,7.882330,1.942559,13.304597,-2.682707,0.623444,0
997,14.421812,-10.688891,5.242771,-2.954794,11.689658,1
998,5.566459,-4.118762,3.670333,7.948329,10.940144,1


#### Define the loss function

In [102]:
def loss_fn(y_true, y_pred):
    # one-hot encoding
    y_true = tf.one_hot(y_true, depth=y_pred.shape[1])
    # convert y_true to double
    y_true = tf.cast(y_true, tf.float64)  

    #calulation using binary cross-entropy
    loss = -(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
    return tf.reduce_mean(loss)

#### Define function to perform prediction

In [103]:
def sigmoid(x):
    # returns S(x) = 1/(1+e^(-x))
    return 1 / (1 + tf.exp(-x)) 

def relu(x):
    # returns (x > y ? x : y)
    return tf.maximum(0, x)

def forward(x, weights, biases):
    """
    Forward propagation function for a two-layer fully connected neural network

    Arguments:
        x (tf.Tensor): Input tensor
        weights (list): List of weight tensors
        biases (list): List of bias tensors

    Returns:
        tf.Tensor: Output tensor.
    """
    layer1 = relu(tf.matmul(x,weights[0]) + biases[0])
    logits = tf.matmul(layer1,weights[1]) + biases[1]
    output = tf.sigmoid(logits)  # add sigmoid activation function
    return output

#### Define function for model training
Display the training and validation loss values for each epoch of the training loop. The displayed value must be in 6 decimal places.<br>
Hint: <br>
Use `tf.GradientTape` to compute the gradients.

In [104]:
def train(x, y, weights, biases, learning_rate):
    """ 
    compute the gradient and update the weights and biases.
    """
    with tf.GradientTape() as tape:
        y_pred = forward(x, weights, biases)
        loss = loss_fn(y, y_pred)
    gradients = tape.gradient(loss, weights + biases)
    for i in range(len(weights)):
        weights[i].assign_sub(learning_rate * gradients[i])
        biases[i].assign_sub(learning_rate * gradients[i+len(weights)])
    return loss

def fit(x_train, y_train, x_val, y_val, weights, biases, learning_rate, epochs):
    """ 
    This function implements the training loop.
    """
    train_loss_results = []
    val_loss_results = []
    for epoch in range(epochs):
        train_loss = train(x_train, y_train, weights, biases, learning_rate)
        train_loss_results.append(train_loss)
        val_loss = loss_fn(y_val, forward(x_val, weights, biases))
        val_loss_results.append(val_loss)
        print(f"Epoch {epoch}: Training loss: {train_loss}, Validation loss: {val_loss}")
    return train_loss_results, val_loss_results

#### Define the tensors to hold the weights and biases (create the model)
Hint: <br>
Use `tf.Variable` to create the tensors.<br>
Put the tensors in a list.

In [105]:
input_size = 5  # 5 features
hidden_size = 3  # Size of the hidden layer
output_size = 2  # Predict 2 classes

# Initialize the weights and biases
weights = [
    tf.Variable(tf.random.normal([input_size, hidden_size], dtype=tf.float64), name='input_layer_weights'),
    tf.Variable(tf.random.normal([hidden_size, output_size], dtype=tf.float64), name='hidden_layer_weights'),
]

biases = [
    tf.Variable(tf.zeros([hidden_size], dtype=tf.float64), name='b1'),
    tf.Variable(tf.zeros([output_size], dtype=tf.float64), name='b2'),
]



#### Split the dataset
The ratio of training and test is 7:1:2.

In [106]:
test_ratio = 0.3
validation_ratio = 1/3

'''
Split dataset into training and test with ratio of 7:3, then split the test set into test and validation with ratio of 1:3
'''

# Split the dataset into training and validation sets
train_data, test_data = train_test_split(df, test_size = test_ratio, random_state=42)

# Split the training and validation data into training and validation sets
test_data, val_data = train_test_split(test_data, test_size= validation_ratio, random_state=42)

# Print the sizes of the three sets
print("Dataset size:", len(df))
print("Training set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))


Dataset size: 1000
Training set size: 700
Validation set size: 100
Test set size: 200


#### Normalize the data

In [107]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
# Initialize the MinMaxScaler
scaler = MinMaxScaler()
#scaler = StandardScaler()

# Fit the scaler to the data and transform the data
#df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Define the input features and target variable
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_val = val_data.drop('label', axis=1)
Y_val = val_data['label']

"""
X_test = test_data.drop('label', axis=1)
Y_test = test_data['label']
"""

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


#### Train the model

In [108]:
# Train the model
train_loss_results, val_loss_results = fit(X_train, y_train, X_val, Y_val, weights, biases, learning_rate=1, epochs=100)


Epoch 0: Training loss: 0.8690236710058024, Validation loss: 0.6637179905613736
Epoch 1: Training loss: 0.674220455262404, Validation loss: 0.6160324373607325
Epoch 2: Training loss: 0.6124586785357938, Validation loss: 0.5949236230282899
Epoch 3: Training loss: 0.5841672160020566, Validation loss: 0.5833425323292017
Epoch 4: Training loss: 0.5711429931165547, Validation loss: 0.5732443274641612
Epoch 5: Training loss: 0.5597362367379498, Validation loss: 0.5628718971102624
Epoch 6: Training loss: 0.5483183844197153, Validation loss: 0.5522354277988216
Epoch 7: Training loss: 0.5369167558404454, Validation loss: 0.5412846031492242
Epoch 8: Training loss: 0.5254892834581424, Validation loss: 0.5300571172563469
Epoch 9: Training loss: 0.5139407059818355, Validation loss: 0.5185991960000886
Epoch 10: Training loss: 0.502287191218439, Validation loss: 0.5069658915743076
Epoch 11: Training loss: 0.4905679023594722, Validation loss: 0.49519904337510945
Epoch 12: Training loss: 0.478853905258

#### Display the training loss and validation loss against epoch graph

#### Predict the test set

#### Display the confusion matrix and the classification report.